### Dataset Flights

Se plantean nuevas hipótesis de datos que ya se limpiaron previamente.

In [15]:
import pandas as pd

df_flights = pd.read_csv('data/flights.csv')

df_flights.loc[df_flights['DepDel15'].isnull(), 'DepDel15'] = df_flights.loc[df_flights['DepDel15'].isnull(), 'DepDelay'].apply(lambda x: 1 if x >= 15 else 0)

#Bounds for DepDelay
Q1_dep = df_flights['DepDelay'].quantile(0.25)
Q3_dep = df_flights['DepDelay'].quantile(0.75)
IQR_dep = Q3_dep - Q1_dep
lower_bound_dep = Q1_dep - 1.5 * IQR_dep
upper_bound_dep = Q3_dep + 1.5 * IQR_dep

#Bounds for ArrDelay
Q1_arr = df_flights['ArrDelay'].quantile(0.25)
Q3_arr = df_flights['ArrDelay'].quantile(0.75)
IQR_arr = Q3_arr - Q1_arr
lower_bound_arr = Q1_arr - 1.5 * IQR_arr
upper_bound_arr = Q3_arr + 1.5 * IQR_arr

#Remove outliers from both DepDelay and ArrDelay
df_flights_cleaned = df_flights[
    (df_flights['DepDelay'] >= lower_bound_dep) & (df_flights['DepDelay'] <= upper_bound_dep) &
    (df_flights['ArrDelay'] >= lower_bound_arr) & (df_flights['ArrDelay'] <= upper_bound_arr)
]

In [16]:
df_flights_cleaned.head()

,Year,Month,DayofMonth,DayOfWeek,Carrier,OriginAirportID,OriginAirportName,OriginCity,OriginState,DestAirportID,DestAirportName,DestCity,DestState,CRSDepTime,DepDelay,DepDel15,CRSArrTime,ArrDelay,ArrDel15,Cancelled
0,2013,9,16,1,DL,15304,Tampa International,Tampa,FL,12478,John F. Kennedy International,New York,NY,1539,4,0.0,1824,13,0,0
1,2013,9,23,1,WN,14122,Pittsburgh International,Pittsburgh,PA,13232,Chicago Midway International,Chicago,IL,710,3,0.0,740,22,1,0
2,2013,9,7,6,AS,14747,Seattle/Tacoma International,Seattle,WA,11278,Ronald Reagan Washington National,Washington,DC,810,-3,0.0,1614,-7,0,0
4,2013,5,16,4,DL,13931,Norfolk International,Norfolk,VA,10397,Hartsfield-Jackson Atlanta International,Atlanta,GA,545,-1,0.0,728,-9,0,0
6,2013,10,6,7,WN,13796,Metropolitan Oakland International,Oakland,CA,12191,William P Hobby,Houston,TX,630,-1,0.0,1210,-3,0,0


## Pruebas de hipótesis

#### **`1. ¿Es el retraso promedio en la salida (DepDelay) mayor a 10 minutos?`**

**1.1 Planteamiento de hipótesis**

- **Hipótesis nula ($H_0$)**: El retraso promedio en la salida es menor o igual a 10 minutos. ($H_0: \mu \leq 10$)
- **Hipótesis alternativa ($H_1$)**: El retraso promedio en la salida es mayor a 10 minutos. ($H_1: \mu > 10$)



In [17]:
# Calcular la media de DepDelay (retraso en la salida)
media_dep_delay = df_flights_cleaned['DepDelay'].mean()
media_dep_delay

np.float64(0.3734467194459452)

**1.2 Prueba t de una muestra**

Aplicamos la **prueba t de Student** usando la siguiente fórmula:

$$
t = \frac{\bar{x} - \mu_0}{\frac{s}{\sqrt{n}}}
$$

Donde:
- $\bar{x}$ es la media muestral (media de los retrasos en la salida).
- $\mu_0$ es la media teórica (10 minutos).
- $s$ es la desviación estándar de la muestra.
- $n$ es el tamaño de la muestra.

Esta fórmula nos da el valor del estadístico t, que usaremos para obtener el valor p (p-value) y decidir si rechazamos o no la hipótesis nula.


In [18]:
from scipy import stats

# Aplicar la prueba t de una muestra
t_statistic, p_value = stats.ttest_1samp(df_flights_cleaned['DepDelay'], 10)

# Mostrar los resultados
t_statistic, p_value


(np.float64(-581.2526354067811), np.float64(0.0))

**1.3 Interpretación del resultado**:

El valor p obtenido nos permitirá rechazar o no la hipótesis nula:
- Si el valor p es menor que 0.05, rechazamos la hipótesis nula y concluimos que el retraso promedio es significativamente mayor a 10 minutos.
- Si el valor p es mayor o igual a 0.05, no podemos rechazar la hipótesis nula.

En este caso, se usará una prueba **unilateral** porque la hipótesis alternativa plantea que el retraso promedio es mayor (no simplemente diferente).


In [19]:
# Interpretación del valor p
if p_value / 2 < 0.05 and t_statistic > 0:
    print("Rechazamos la hipótesis nula: el retraso promedio es significativamente mayor a 10 minutos.")
else:
    print("No rechazamos la hipótesis nula: no hay evidencia suficiente para decir que el retraso promedio es mayor a 10 minutos.")


No rechazamos la hipótesis nula: no hay evidencia suficiente para decir que el retraso promedio es mayor a 10 minutos.


#### **`2. ¿Es el retraso promedio en la llegada (ArrDelay) menor o igual a 5 minutos?`**




**2.1 Planteamiento de hipótesis**

- **Hipótesis nula ($H_0$)**: El retraso promedio en la llegada es menor o igual a 5 minutos. ($H_0: \mu \leq 5$)
- **Hipótesis alternativa ($H_1$)**: El retraso promedio en la llegada es mayor a 5 minutos. ($H_1: \mu > 5$)

En este caso, queremos evaluar si el retraso promedio en la llegada de los vuelos es mayor a 5 minutos.

In [20]:
# Calcular la media de ArrDelay (retraso en la llegada)
media_arr_delay = df_flights_cleaned['ArrDelay'].mean()
media_arr_delay

np.float64(-4.029842555526722)

**2.2Prueba t de una muestra**

Aplicamos la **prueba t de Student** usando la siguiente fórmula:

$$
t = \frac{\bar{x} - \mu_0}{\frac{s}{\sqrt{n}}}
$$

Donde:
- $\bar{x}$ es la media muestral (media de los retrasos en la llegada).
- $\mu_0$ es la media teórica (5 minutos).
- $s$ es la desviación estándar de la muestra.
- $n$ es el tamaño de la muestra.

Esta fórmula nos da el valor del estadístico t, que usaremos para obtener el valor p (p-value) y decidir si rechazamos o no la hipótesis nula.


In [21]:
from scipy import stats

# Aplicar la prueba t de una muestra
t_statistic, p_value = stats.ttest_1samp(df_flights_cleaned['ArrDelay'], 5)

# Mostrar los resultados
t_statistic, p_value


(np.float64(-337.8163162820931), np.float64(0.0))

**2.3 Interpretación del resultado**:

El valor p obtenido nos permitirá rechazar o no la hipótesis nula:
- Si el valor p es menor que 0.05, rechazamos la hipótesis nula y concluimos que el retraso promedio en la llegada es significativamente mayor a 5 minutos.
- Si el valor p es mayor o igual a 0.05, no podemos rechazar la hipótesis nula.

En este caso, se usará una prueba **unilateral** porque la hipótesis alternativa plantea que el retraso promedio es mayor (no simplemente diferente).


In [22]:
# Interpretación del valor p
if p_value / 2 < 0.05 and t_statistic > 0:
    print("Rechazamos la hipótesis nula: el retraso promedio en la llegada es significativamente mayor a 5 minutos.")
else:
    print("No rechazamos la hipótesis nula: no hay evidencia suficiente para decir que el retraso promedio es mayor a 5 minutos.")


No rechazamos la hipótesis nula: no hay evidencia suficiente para decir que el retraso promedio es mayor a 5 minutos.


#### **`3. ¿Es la desviación estándar de los retrasos en la salida (DepDelay) mayor que 12 minutos?`**

**3.1 Planteamiento de hipótesis**

- **Hipótesis nula ($H_0$)**: La desviación estándar de los retrasos en la salida es menor o igual a 12 minutos. ($H_0: \sigma \leq 12$)
- **Hipótesis alternativa ($H_1$)**: La desviación estándar de los retrasos en la salida es mayor a 12 minutos. ($H_1: \sigma > 12$)

En este caso, queremos evaluar si la desviación estándar de los retrasos en la salida de los vuelos es mayor a 12 minutos.


In [23]:
# Calcular la desviación estándar de DepDelay (retraso en la salida)
std_dep_delay = df_flights_cleaned['DepDelay'].std()
std_dep_delay


np.float64(8.029744308838586)

**3.2 Prueba chi-cuadrada**

Aplicamos la **prueba chi-cuadrada** para la varianza usando la fórmula:

$$
\chi^2 = \frac{(n-1) \cdot s^2}{\sigma_0^2}
$$

Donde:
- $n$ es el tamaño de la muestra.
- $s^2$ es la varianza muestral (la desviación estándar muestral al cuadrado).
- $\sigma_0^2$ es la varianza teórica basada en 12 minutos (12^2 = 144).

Esta fórmula nos da el valor del estadístico chi-cuadrado, que se compara con una distribución chi-cuadrada con $n-1$ grados de libertad para obtener el valor p.



In [24]:
import numpy as np
from scipy.stats import chi2

# Número de muestras (n)
n = len(df_flights_cleaned['DepDelay'])

# Varianza muestral
var_muestral = std_dep_delay**2

# Varianza teórica (12^2 = 144)
var_teorica = 12**2

# Estadístico chi-cuadrado
chi_square_stat = (n - 1) * var_muestral / var_teorica

# Grados de libertad
gl = n - 1

# Obtener el valor p para una prueba unilateral (mayor que)
p_value = 1 - chi2.cdf(chi_square_stat, gl)

# Mostrar el estadístico chi-cuadrado y el valor p
chi_square_stat, p_value


(np.float64(105252.09708148224), np.float64(1.0))

**3.3 Interpretación del resultado**:

Con base en el valor p obtenido, tomamos una decisión sobre la hipótesis nula.

In [25]:
# Interpretación del valor p
if p_value < 0.05:
    print("Rechazamos la hipótesis nula: la desviación estándar de los retrasos en la salida es significativamente mayor a 12 minutos.")
else:
    print("No rechazamos la hipótesis nula: no hay evidencia suficiente para decir que la desviación estándar de los retrasos en la salida es mayor a 12 minutos.")


No rechazamos la hipótesis nula: no hay evidencia suficiente para decir que la desviación estándar de los retrasos en la salida es mayor a 12 minutos.
